# All Model saves here
- option2 : user separate 2:1 = train : val do not overlap dataset

## import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
import DeepMIMOv3
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import time
import math
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import IterableDataset
import numpy as np
import time, gc
from tqdm import tqdm
import numpy as np
import torch
import random
import torch.nn as nn
from lwm_model import lwm
from torch.optim import Adam
from pathlib import Path
import torch, time



## GPU Settings

In [2]:
# GPU 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
import torch
print(torch.version.cuda)                   
print(torch.backends.cudnn.version())       
print("CUDA available:", torch.cuda.is_available())  # True

12.6
90501
CUDA available: True


## DeepMIMOv3 dataset

In [4]:
parameters = DeepMIMOv3.default_params()

In [5]:
## Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
#LWM dynamic senario
# parameters['dataset_folder'] = r'/content/drive/MyDrive/Colab Notebooks/LWM'
scene = 15 # scene 15
# change my linux route
parameters['dataset_folder'] = '/home/dlghdbs200/LWM'

# scnario = 02_dyn_3p5 <- download file
parameters['scenario'] = 'O2_dyn_3p5'
parameters['dynamic_scenario_scenes'] = np.arange(scene) #scene 0~9

# Up to 10 multipath paths per user-to-base station channel
parameters['num_paths'] = 10

# User rows 1-100
parameters['user_rows'] = np.arange(100)
# User subsampling
parameters['user_subsampling'] = 0.01

# Activate only the first basestation
parameters['active_BS'] = np.array([1])

parameters['activate_OFDM'] = 1

parameters['OFDM']['bandwidth'] = 0.05 # 50 MHz
parameters['OFDM']['subcarriers'] = 512 # OFDM with 512 subcarriers
parameters['OFDM']['selected_subcarriers'] = np.arange(0, 64, 1)
#parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['ue_antenna']['shape'] = np.array([1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32]) # ULA of 32 elements
#parameters['bs_antenna']['rotation'] = np.array([0, 30, 90]) # ULA of 32 elements
#parameters['ue_antenna']['rotation'] = np.array([[0, 30], [30, 60], [60, 90]]) # ULA of 32 elements
#parameters['ue_antenna']['radiation_pattern'] = 'isotropic'
#parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [6]:
## dataset setting (chunked on‑the‑fly generation)
import time, gc
from tqdm import tqdm

# 0~999 scene index , process 50 at that time
scene_indices = np.arange(scene)
chunk_size   = 5
all_data     = []

# Call generate_data for each scene chunk
for i in tqdm(range(0, len(scene_indices), chunk_size)):
    chunk = scene_indices[i : i+chunk_size].tolist()
    parameters['dynamic_scenario_scenes'] = chunk

    start = time.time()
    data_chunk = DeepMIMOv3.generate_data(parameters)
    print(f"Scenes {chunk[0]}–{chunk[-1]} generation time: {time.time() - start:.2f}s")

    # combine all_data or save in the Disk
    all_data.extend(data_chunk)

    # free memory 
    del data_chunk
    gc.collect()

# comvine Dataset
dataset = all_data


print(parameters['user_rows'])

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]

The following parameters seem unnecessary:
{'activate_OFDM'}

Scene 1/5

Basestation 1

UE-BS Channels



Reading ray-tracing: 100%|████████████████████████████████████████████████████| 69006/69006 [00:00<00:00, 345614.96it/s]

Generating channels: 100%|██████████████████████████████████████████████████████████| 727/727 [00:00<00:00, 8464.07it/s]



BS-BS Channels



Reading ray-tracing: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 6636.56it/s]

Generating channels: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 910.82it/s]


...


Scene 5/5

Basestation 1

UE-BS Channels



Reading ray-tracing: 100%|████████████████████████████████████████████████████| 69006/69006 [00:00<00:00, 315554.02it/s]

Generating channels: 100%|██████████████████████████████████████████████████████████| 727/727 [00:00<00:00, 7882.58it/s]



BS-BS Channels



Reading ray-tracing: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 7194.35it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:19<00:00,  6.62s/it]

Scenes 10–14 generation time: 6.55s
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


## do not overlap dataset and separate train : val = 2 : 1

In [7]:

# all User
U = dataset[0][0]['user']['channel'].shape[0]   # ex) 737

# separate 2:1 = train : val
user_ids = np.arange(U)
random.shuffle(user_ids)          
cut = int(len(user_ids) * 2 / 3)

train_users = set(user_ids[:cut])   # 2/3 → Train
val_users   = set(user_ids[cut:])   # 1/3 → Val
# ────────────────────────────────────────────────────────────────



## About Information
User : 737
UE antenna : 1
BS antenna : 32  Shape(a+bj)
subcarrier : 64

In [8]:
# Unmasked Data Model(gru
# separate maksed data and unmasked data

## Data Preprocessing

In [9]:
# =============================================================================
# UnMaskedChannelSeqDataset
#   • Predict next-step channel vector from past `seq_len` steps (no masking)
#   • Supports user-level Train / Val split via `user_filter`
#   • Power-normalises complex channel → real + imag concat, then Min–Max scales
# =============================================================================
from typing import Optional, Set, Tuple

import numpy as np
import torch
from torch.utils.data import IterableDataset
from sklearn.preprocessing import MinMaxScaler


class UnMaskedChannelSeqDataset(IterableDataset):
    """
    IterableDataset (un-masked version).

    Args
    ----
    scenes : list
        DeepMIMO scene dictionaries.
    seq_len : int
        Number of past time-steps used as input.
    eps : float
        Small constant to avoid division by zero in power normalisation.
    scalers : tuple(MinMaxScaler, MinMaxScaler) | None
        Pre-fitted (x, y) scalers.  If None, fit scalers on *this* dataset.
    user_filter : set[int] | None
        If given, only yield samples for those user indices.
    """
    def __init__(
        self,
        scenes,
        seq_len: int = 5,
        eps: float = 1e-9,
        scalers: Optional[Tuple[MinMaxScaler, MinMaxScaler]] = None,
        user_filter: Optional[Set[int]] = None,
    ):
        super().__init__()
        self.scenes      = scenes
        self.seq_len     = seq_len
        self.eps         = eps
        self.user_filter = user_filter

        # Channel tensor dimensions -------------------------------------------------
        ch0          = scenes[0][0]['user']['channel']   # (U, 1, A, S)
        self.U       = ch0.shape[0]                      # users
        self.A       = ch0.shape[2]                      # BS antennas
        self.S       = ch0.shape[3]                      # sub-carriers
        self.vec_len = 2 * self.A                       # real + imag concatenation

        # Fit / reuse MinMax scalers ------------------------------------------------
        if scalers is None:
            X_list, y_list = [], []
            T = len(scenes)
            for t in range(self.seq_len, T):
                past  = scenes[t - self.seq_len : t]
                s_tgt = scenes[t]

                for u in range(self.U):
                    if self.user_filter is not None and u not in self.user_filter:
                        continue
                    for s in range(self.S):
                        seq_np = np.stack(
                            [self._power_norm(p[0]['user']['channel'][u, 0, :, s])
                             for p in past],
                            axis=0, dtype=np.float32
                        )
                        tgt_np = self._power_norm(
                            s_tgt[0]['user']['channel'][u, 0, :, s]
                        ).astype(np.float32)

                        if not np.any(seq_np) or not np.any(tgt_np):
                            continue

                        X_list.append(seq_np.reshape(-1, self.vec_len))
                        y_list.append(tgt_np)

            X_all = np.vstack(X_list)
            y_all = np.stack(y_list)
            self.scaler_x = MinMaxScaler().fit(X_all)
            self.scaler_y = MinMaxScaler().fit(y_all)
        else:
            self.scaler_x, self.scaler_y = scalers

    # -----------------------------------------------------------------------------  
    # Iterator
    # -----------------------------------------------------------------------------
    def __iter__(self):
        T = len(self.scenes)
        for t in range(self.seq_len, T):
            past  = self.scenes[t - self.seq_len : t]
            s_tgt = self.scenes[t]

            for u in range(self.U):
                if self.user_filter is not None and u not in self.user_filter:
                    continue
                for s in range(self.S):
                    seq_np = np.stack(
                        [self._power_norm(p[0]['user']['channel'][u, 0, :, s])
                         for p in past],
                        axis=0
                    )
                    tgt_np = self._power_norm(
                        s_tgt[0]['user']['channel'][u, 0, :, s]
                    )

                    if not np.any(seq_np) or not np.any(tgt_np):
                        continue

                    N, D = seq_np.shape
                    seq_np = self.scaler_x.transform(seq_np.reshape(-1, D)).reshape(N, D)
                    tgt_np = self.scaler_y.transform(tgt_np.reshape(1, -1)).reshape(-1,)

                    yield (
                        torch.from_numpy(seq_np).float(),  # (seq_len, vec_len)
                        torch.from_numpy(tgt_np).float()   # (vec_len,)
                    )

    # -----------------------------------------------------------------------------  
    # Helpers
    # -----------------------------------------------------------------------------
    def _power_norm(self, h: np.ndarray) -> np.ndarray:
        """Convert complex vector → real|imag concat, then normalise power to 1."""
        v     = np.concatenate([h.real, h.imag]).astype(np.float32)
        power = np.mean(v * v) + self.eps
        return v / np.sqrt(power)

    def __len__(self):
        """Rough size estimate (IterableDataset doesn't rely on this)."""
        return (len(self.scenes) - self.seq_len) * self.U * self.S


In [10]:
# =============================================================================
# MaskedChannelSeqDataset
#   • Same preprocessing as UnMasked, plus 15 % BERT-style masking on sequences
#   • Masking policy:
#       80 % → replace with zeros
#       10 % → replace with Gaussian noise
#       10 % → keep original patch (label remains the same)
# =============================================================================

class MaskedChannelSeqDataset(IterableDataset):
    """
    IterableDataset (masked version).

    Additional Args
    ---------------
    noise_std : float
        Standard deviation for Gaussian noise patches.
    user_filter : set[int] | None
        If given, only yield samples for those user indices.
    """
    def __init__(
        self,
        scenes,
        seq_len: int = 5,
        eps: float = 1e-9,
        noise_std: float = 1.0,
        user_filter: Optional[Set[int]] = None,
    ):
        super().__init__()
        self.scenes      = scenes
        self.seq_len     = seq_len
        self.eps         = eps
        self.noise_std   = noise_std
        self.user_filter = user_filter

        # Channel tensor dimensions -------------------------------------------------
        ch0          = scenes[0][0]['user']['channel']
        self.U       = ch0.shape[0]
        self.A       = ch0.shape[2]
        self.S       = ch0.shape[3]
        self.vec_len = 2 * self.A

        # Fit scalers on power-normalised data --------------------------------------
        X_list, y_list = [], []
        T = len(scenes)
        for t in range(self.seq_len, T):
            past  = scenes[t - self.seq_len : t]
            s_tgt = scenes[t]
            for u in range(self.U):
                if self.user_filter is not None and u not in self.user_filter:
                    continue
                for s in range(self.S):
                    seq_np = np.stack(
                        [self._power_norm(p[0]['user']['channel'][u, 0, :, s])
                         for p in past],
                        axis=0, dtype=np.float32
                    )
                    tgt_np = self._power_norm(
                        s_tgt[0]['user']['channel'][u, 0, :, s]
                    ).astype(np.float32)

                    if not np.any(seq_np) or not np.any(tgt_np):
                        continue

                    X_list.append(seq_np.reshape(-1, self.vec_len))
                    y_list.append(tgt_np)

        X_all = np.vstack(X_list)
        y_all = np.stack(y_list)
        self.scaler_x = MinMaxScaler().fit(X_all)
        self.scaler_y = MinMaxScaler().fit(y_all)

        # Pre-allocated zero mask
        self.mask_value = torch.zeros(self.vec_len, dtype=torch.float32)

    # -----------------------------------------------------------------------------  
    # Iterator
    # -----------------------------------------------------------------------------
    def __iter__(self):
        mask_prob  = 0
        zero_prob  = mask_prob * 0.8
        noise_prob = mask_prob * 0.1

        T = len(self.scenes)
        for t in range(self.seq_len, T):
            past  = self.scenes[t - self.seq_len : t]
            s_tgt = self.scenes[t]

            for u in range(self.U):
                if self.user_filter is not None and u not in self.user_filter:
                    continue
                for s in range(self.S):
                    seq_np = np.stack(
                        [self._power_norm(p[0]['user']['channel'][u, 0, :, s])
                         for p in past],
                        axis=0
                    )
                    tgt_np = self._power_norm(
                        s_tgt[0]['user']['channel'][u, 0, :, s]
                    )

                    if not np.any(seq_np) or not np.any(tgt_np):
                        continue

                    N, D = seq_np.shape
                    seq_np = self.scaler_x.transform(seq_np.reshape(-1, D)).reshape(N, D)
                    tgt_np = self.scaler_y.transform(tgt_np.reshape(1, -1)).reshape(-1,)

                    seq_tensor   = torch.from_numpy(seq_np).float()
                    target_tensor= torch.from_numpy(tgt_np).float()

                    mpos = np.random.randint(self.seq_len)
                    r    = np.random.rand()

                    if r < zero_prob:
                        masked_seq = seq_tensor.clone()
                        masked_seq[mpos] = self.mask_value
                        yield masked_seq, torch.tensor([mpos]), target_tensor

                    elif r < zero_prob + noise_prob:
                        masked_seq = seq_tensor.clone()
                        masked_seq[mpos] = torch.randn(self.vec_len) * self.noise_std
                        yield masked_seq, torch.tensor([mpos]), target_tensor

                    elif r < mask_prob:
                        yield seq_tensor, torch.tensor([mpos]), target_tensor  # kept patch

                    else:  # no masking
                        yield seq_tensor, torch.tensor([mpos]), target_tensor

    # -----------------------------------------------------------------------------  
    # Helpers
    # -----------------------------------------------------------------------------
    def _power_norm(self, h: np.ndarray) -> np.ndarray:
        v     = np.concatenate([h.real, h.imag]).astype(np.float32)
        power = np.mean(v * v) + self.eps
        return v / np.sqrt(power)

    def __len__(self):
        return (len(self.scenes) - self.seq_len) * self.U * self.S


## define trainable parameters

In [11]:
def count_trainable_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Split Train/Val

In [12]:
seq_len   = 5
ratio     = 0.6          # 6 : 4
batch_size= 32

# 1) Derive user-ID sets --------------------------------------------------------
U = dataset[0][0]['user']['channel'].shape[0]   # e.g. 737
user_ids = np.arange(U)
random.shuffle(user_ids)                        # reproducible → random.seed(42)
cut = int(U * ratio)

train_users = set(user_ids[:cut])   # 60 %
val_users   = set(user_ids[cut:])   # 40 %


In [13]:
# 2) Un-masked datasets  (share scaler to avoid leakage) -----------------------
unmasked_train_ds = UnMaskedChannelSeqDataset(
    scenes      = dataset,
    seq_len     = seq_len,
    user_filter = train_users
)

unmasked_val_ds = UnMaskedChannelSeqDataset(
    scenes      = dataset,
    seq_len     = seq_len,
    scalers     = (unmasked_train_ds.scaler_x,   # reuse train scalers
                   unmasked_train_ds.scaler_y),
    user_filter = val_users
)

unmasked_train_loader = DataLoader(unmasked_train_ds, batch_size=batch_size, shuffle=False)
unmasked_val_loader   = DataLoader(unmasked_val_ds,   batch_size=batch_size, shuffle=False)

In [14]:
# 3) Masked datasets -----------------------------------------------------------
masked_train_ds = MaskedChannelSeqDataset(
    scenes      = dataset,
    seq_len     = seq_len,
    user_filter = train_users
)

masked_val_ds = MaskedChannelSeqDataset(
    scenes      = dataset,
    seq_len     = seq_len,
    user_filter = val_users
)

masked_train_loader = DataLoader(masked_train_ds, batch_size=batch_size, shuffle=False)
masked_val_loader   = DataLoader(masked_val_ds,   batch_size=batch_size, shuffle=False)
# ─────────────────────────────────────────────

## Define Model

LWMWithHead: A wrapper class that uses a pre-trained LWM (Transformer encoder) as the backbone,
             and attaches a new fully-connected (FC) head for downstream tasks
             (regression, classification, etc.).

Changes:
- input_dim: Dimension of the actual input data (e.g., 64)
- patch_length: Patch length expected by the backbone (e.g., 16)
- Replaces the original element_length parameter with these two distinct parameters
- Applies a projection layer (self.input_proj) in forward()


In [15]:
class LWMWithHead(nn.Module):
    """
    LWMWithHead: A wrapper class that uses a pre-trained LWM (Transformer encoder) as the backbone,
                 and attaches a new fully-connected (FC) head for downstream tasks
                 (regression, classification, etc.).

    Changes:
    - input_dim: Dimension of the actual input data (e.g., 64)
    - patch_length: Patch length expected by the backbone (e.g., 16)
    - Replaces the original element_length parameter with these two distinct parameters
    - Applies a projection layer (self.input_proj) in forward()
    """
    def __init__(
        self,
        input_dim: int,                 # Dimension of the actual input data (e.g., 64)
        patch_length: int,              # Patch length expected by the backbone (e.g., 16)
        d_model: int = 64,              # LWM hidden size
        max_len: int = 129,             # Positional encoding max length
        n_layers: int = 12,             # Number of Transformer encoder layers
        hidden_dim: int = 256,          # FC head hidden dimension
        out_dim: int = 64,              # FC head output dimension
        freeze_backbone: bool = True,   # Whether to freeze the backbone
        checkpoint_path: str | None = "./model_weights.pth",
        device: str = "cuda"
    ):
        super().__init__()

        # apply a projection layer to match backbone's expected patch_length
        self.input_proj = nn.Linear(input_dim, patch_length)

        # initialize backbone
        if checkpoint_path is None:
            # randomly initialized backbone
            self.backbone = lwm(
                element_length=patch_length,
                d_model=d_model,
                max_len=max_len,
                n_layers=n_layers
            ).to(device)
        else:
            # load pre-trained weights
            self.backbone = lwm.from_pretrained(
                ckpt_name=checkpoint_path,
                device=device
            )

        # freeze backbone parameters if required
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        # attach a new fully-connected head for downstream tasks
        self.head = nn.Sequential(
            # change 2 layer -> 1 layer
            nn.Linear(d_model, out_dim),
        )

    def forward(self, input_ids: torch.Tensor, masked_pos: torch.Tensor) -> torch.Tensor:
        """
        Args:
            input_ids: Tensor of shape (B, L, input_dim)
            masked_pos: Tensor of shape (B, num_mask)
        Returns:
            out: Tensor of shape (B, out_dim)
        """
        # project inputs to patch_length dimension
        x = self.input_proj(input_ids)

        # backbone forward: returns (logits_lm, enc_output)
        _, enc_output = self.backbone(x, masked_pos)

        # extract CLS token feature (first token)
        feat = enc_output[:, 0, :]

        # pass through FC head to get final output
        out = self.head(feat)
        return out


In [16]:
import torch
import torch.nn as nn

class GRUWithHead(nn.Module):
    """
    GRUWithHead (projected):
      • Projects the raw feature dimension (input_dim) to a smaller patch_length
        so every backbone receives the same patch-sized input (like LWM).
      • Stacks N GRU layers, then an FC head for downstream tasks.
    """
    def __init__(
        self,
        input_dim: int    = 64,   # raw feature dimension coming from the DataLoader
        patch_length: int = 16,   # target dimension fed to the GRU backbone
        d_model: int      = 64,   # GRU hidden size
        n_layers: int     = 12,   # number of stacked GRU layers
        bidirectional: bool = True,
        dropout: float      = 0.1,
        hidden_dim: int     = 256, # FC-head hidden size
        out_dim: int        = 64,  # FC-head output size
        freeze_backbone: bool = False
    ):
        super().__init__()

        # 0) Project raw_dim → patch_length (64 → 16)
        self.input_proj = nn.Linear(input_dim, patch_length)

        # 1) GRU backbone that expects 'patch_length' features per time step
        self.backbone = nn.GRU(
            input_size     = patch_length,
            hidden_size    = d_model,
            num_layers     = n_layers,
            batch_first    = True,
            bidirectional  = bidirectional,
            dropout        = dropout if n_layers > 1 else 0.0
        )

        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        # 2) Fully-connected head
        gru_out_dim = d_model * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.Linear(gru_out_dim, out_dim),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x : Tensor of shape (batch, seq_len, input_dim) – raw features
        Returns:
            Tensor of shape (batch, out_dim)
        """
        # project raw features to patch_length
        x_proj = self.input_proj(x)                 # (B, seq_len, patch_length)

        # sequence modelling with GRU
        out, _ = self.backbone(x_proj)              # (B, seq_len, num_dirs*d_model)

        # use the last time-step representation
        feat = out[:, -1, :]                        # (B, gru_out_dim)

        # downstream head
        return self.head(feat)                      # (B, out_dim)


In [17]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        # Create positional encoding matrix of shape (1, max_len, d_model)
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor of shape (batch_size, seq_len, d_model)
        Returns:
            Tensor: x plus positional encodings
        """
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

class InputEmbedding(nn.Module):
    def __init__(self, feat_dim: int, d_model: int, max_len: int = 5000):
        super().__init__()
        # Optional linear projection from feat_dim to d_model
        self.proj = nn.Linear(feat_dim, d_model) if feat_dim != d_model else None
        self.pos_enc = PositionalEncoding(d_model, max_len)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor of shape (batch, seq_len, feat_dim)
        Returns:
            Tensor of shape (batch, seq_len, d_model)
        """
        if self.proj is not None:
            x = self.proj(x)
        return self.pos_enc(x)

class EncoderLayer(nn.Module):
    def __init__(self, d_model: int, n_heads: int, dim_ff: int, dropout: float = 0.1):
        super().__init__()
        # Multi-Head Self-Attention
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        # Position-wise Feed-Forward Network
        self.ff = nn.Sequential(
            nn.Linear(d_model, dim_ff),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(dim_ff, d_model)
        )
        # Layer Normalization and Dropout for residual connections
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(
        self,
        x: torch.Tensor,
        src_mask: torch.Tensor = None,
        src_key_padding_mask: torch.Tensor = None
    ) -> torch.Tensor:
        """
        Args:
            x: Tensor of shape (seq_len, batch, d_model)
            src_mask: Optional Tensor of shape (seq_len, seq_len)
            src_key_padding_mask: Optional Tensor of shape (batch, seq_len)
        Returns:
            Tensor of shape (seq_len, batch, d_model)
        """
        # Self-attention sublayer
        attn_out, _ = self.self_attn(x, x, x, attn_mask=src_mask, key_padding_mask=src_key_padding_mask)
        x = x + self.dropout1(attn_out)
        x = self.norm1(x)
        # Feed-forward sublayer
        ff_out = self.ff(x)
        x = x + self.dropout2(ff_out)
        x = self.norm2(x)
        return x

class TransformerEncoderCustom(nn.Module):
    def __init__(
        self,
        feat_dim: int,
        d_model: int,
        n_heads: int,
        dim_ff: int,
        n_layers: int,
        dropout: float = 0.1,
        max_len: int = 5000
    ):
        super().__init__()
        # Input embedding: feature projection + positional encoding
        self.input_embedding = InputEmbedding(feat_dim, d_model, max_len)
        # Stack of N encoder layers
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, dim_ff, dropout)
            for _ in range(n_layers)
        ])

    def forward(
        self,
        x: torch.Tensor,
        src_mask: torch.Tensor = None,
        src_key_padding_mask: torch.Tensor = None
    ) -> torch.Tensor:
        """
        Args:
            x: Tensor of shape (batch, seq_len, feat_dim)
        Returns:
            Tensor of shape (seq_len, batch, d_model)
        """
        x = self.input_embedding(x)       # (batch, seq_len, d_model)
        x = x.transpose(0, 1)             # (seq_len, batch, d_model)
        for layer in self.layers:
            x = layer(x, src_mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model: int, n_heads: int, dim_ff: int, dropout: float = 0.1):
        super().__init__()
        # Masked Self-Attention
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        # Encoder-Decoder Attention
        self.multihead_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        # Position-wise Feed-Forward Network
        self.ff = nn.Sequential(
            nn.Linear(d_model, dim_ff),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(dim_ff, d_model)
        )
        # Layer Normalizations and Dropouts
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(
        self,
        tgt: torch.Tensor,
        memory: torch.Tensor,
        tgt_mask: torch.Tensor = None,
        memory_mask: torch.Tensor = None,
        tgt_key_padding_mask: torch.Tensor = None,
        memory_key_padding_mask: torch.Tensor = None
    ) -> torch.Tensor:
        """
        Args:
            tgt: Tensor of shape (tgt_len, batch, d_model)
            memory: Tensor of shape (src_len, batch, d_model)
        Returns:
            Tensor of shape (tgt_len, batch, d_model)
        """
        # Masked self-attention sublayer
        attn1, _ = self.self_attn(
            tgt, tgt, tgt,
            attn_mask=tgt_mask,
            key_padding_mask=tgt_key_padding_mask
        )
        tgt = tgt + self.dropout1(attn1)
        tgt = self.norm1(tgt)
        # Encoder-decoder attention sublayer
        attn2, _ = self.multihead_attn(
            tgt, memory, memory,
            attn_mask=memory_mask,
            key_padding_mask=memory_key_padding_mask
        )
        tgt = tgt + self.dropout2(attn2)
        tgt = self.norm2(tgt)
        # Feed-forward sublayer
        ff_out = self.ff(tgt)
        tgt = tgt + self.dropout3(ff_out)
        tgt = self.norm3(tgt)
        return tgt

class TransformerDecoderCustom(nn.Module):
    def __init__(
        self,
        feat_dim: int,
        d_model: int,
        n_heads: int,
        dim_ff: int,
        n_layers: int,
        dropout: float = 0.1,
        max_len: int = 5000
    ):
        super().__init__()
        # Input embedding for target sequence
        self.input_embedding = InputEmbedding(feat_dim, d_model, max_len)
        # Stack of N decoder layers
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, dim_ff, dropout)
            for _ in range(n_layers)
        ])
        # Final projection back to feature dimension
        # self.output_linear = nn.Linear(d_model, feat_dim)
        self.output_linear = nn.Identity()

    def forward(
        self,
        tgt: torch.Tensor,
        memory: torch.Tensor,
        tgt_mask: torch.Tensor = None,
        memory_mask: torch.Tensor = None,
        tgt_key_padding_mask: torch.Tensor = None,
        memory_key_padding_mask: torch.Tensor = None
    ) -> torch.Tensor:
        """
        Args:
            tgt: Tensor of shape (batch, tgt_len, feat_dim)
            memory: Tensor of shape (src_len, batch, d_model)
        Returns:
            Tensor of shape (batch, tgt_len, feat_dim)
        """
        x = self.input_embedding(tgt)       # (batch, tgt_len, d_model)
        x = x.transpose(0, 1)               # (tgt_len, batch, d_model)
        for layer in self.layers:
            x = layer(
                x,
                memory,
                tgt_mask=tgt_mask,
                memory_mask=memory_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=memory_key_padding_mask
            )
        x = x.transpose(0, 1)               # (batch, tgt_len, d_model)
        return self.output_linear(x)        # project back to feat_dim

        

class TransformerWithHead(nn.Module):
    def __init__(
        self,
        input_dim: int    = 64,   # raw feature dimension
        patch_length: int = 16,   # sequence length consumed by encoder/decoder
        d_model: int      = 64,   # hidden size inside the transformer
        n_heads: int      = 4,
        dim_ff: int       = 256,
        n_layers: int     = 12,
        dropout: float    = 0.1,
        hidden_dim: int   = 256,
        out_dim: int      = 64,
        max_len: int      = 5000,
        freeze_backbone: bool = False,
    ):
        super().__init__()

        # 0) Project raw input dimension to patch length
        self.input_proj = nn.Linear(input_dim, patch_length)

        # 1) Encoder: processes the source sequence
        self.encoder = TransformerEncoderCustom(
            feat_dim = patch_length,
            d_model  = d_model,
            n_heads  = n_heads,
            dim_ff   = dim_ff,
            n_layers = n_layers,
            dropout  = dropout,
            max_len  = max_len,
        )
        if freeze_backbone:
            for p in self.encoder.parameters():
                p.requires_grad = False

        # 2) Decoder: generates target sequence using encoder memory
        self.decoder = TransformerDecoderCustom(
            feat_dim = patch_length,
            d_model  = d_model,
            n_heads  = n_heads,
            dim_ff   = dim_ff,
            n_layers = n_layers,
            dropout  = dropout,
            max_len  = max_len,
        )

        # 3) Task head: maps final decoder output to desired output dimension
        self.head = nn.Sequential(
            nn.Linear(d_model, out_dim)
        )

    def forward(
        self,
        src: torch.Tensor,                # (batch, src_len, input_dim)
        tgt: torch.Tensor,                # (batch, tgt_len, input_dim)
        src_mask: torch.Tensor = None,
        src_key_padding_mask: torch.Tensor = None,
        tgt_mask: torch.Tensor = None,
        tgt_key_padding_mask: torch.Tensor = None,
    ) -> torch.Tensor:
        # 1) Encode source sequence to produce memory
        src_patch = self.input_proj(src)  # (batch, src_len, patch_length)
        memory = self.encoder(
            src_patch,
            src_mask=src_mask,
            src_key_padding_mask=src_key_padding_mask
        )  # (src_len, batch, d_model)

        # 2) Decode target sequence using encoder memory
        tgt_patch = self.input_proj(tgt)  # (batch, tgt_len, patch_length)
        dec_out = self.decoder(
            tgt_patch,
            memory,
            tgt_mask=tgt_mask,
            memory_mask=None,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask
        )  # (batch, tgt_len, d_model)

        # 3) Use last time-step output from decoder for prediction
        last_step = dec_out[:, -1, :]      # (batch, d_model)
        return self.head(last_step)        # (batch, out_dim)


In [18]:
class RNNWithHead(nn.Module):
    """
    RNNWithHead (projected):
      • Projects raw feature vectors from `input_dim` to `patch_length`
      • Feeds the projected sequence to an RNN backbone
      • Maps the last hidden state through an FC head
    """
    def __init__(
        self,
        input_dim: int    = 64,   # raw feature dimension coming from DataLoader
        patch_length: int = 16,   # dimension consumed by the RNN backbone
        hidden_size: int  = 64,   # RNN hidden size
        num_layers: int   = 12,   # number of stacked RNN layers
        bidirectional: bool = True,
        dropout: float      = 0.1,
        hidden_dim: int     = 256, # FC-head hidden size
        out_dim: int        = 64,  # FC-head output size
        freeze_backbone: bool = False,
    ):
        super().__init__()

        # 0) project raw 64-dim → 16-dim
        self.input_proj = nn.Linear(input_dim, patch_length)

        # 1) RNN backbone
        self.backbone = nn.RNN(
            input_size     = patch_length,
            hidden_size    = hidden_size,
            num_layers     = num_layers,
            batch_first    = True,
            bidirectional  = bidirectional,
            dropout        = dropout if num_layers > 1 else 0.0,
        )

        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        # 2) FC head
        rnn_out_dim = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.Linear(rnn_out_dim, out_dim)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (batch, seq_len, input_dim=64)
        returns: (batch, out_dim)
        """
        x_proj = self.input_proj(x)           # (batch, seq_len, 16)
        out, _ = self.backbone(x_proj)        # (batch, seq_len, rnn_out_dim)
        feat   = out[:, -1, :]                # take last time step
        return self.head(feat)                # (batch, out_dim)


In [19]:
class LSTMWithHead(nn.Module):
    """
    LSTMWithHead (projected):
      • Projects raw feature vectors from `input_dim` to a compact `patch_length`
      • Feeds the projected sequence to an LSTM backbone
      • Uses the last hidden state to drive an FC head for the downstream task
    """
    def __init__(
        self,
        input_dim: int    = 64,   # raw feature dimension (e.g., 64)
        patch_length: int = 16,   # dimension consumed by the LSTM backbone
        hidden_size: int  = 64,   # LSTM hidden size
        num_layers: int   = 12,   # number of stacked LSTM layers
        bidirectional: bool = True,
        dropout: float      = 0.1,
        hidden_dim: int     = 256, # FC-head hidden size
        out_dim: int        = 64,  # FC-head output size
        freeze_backbone: bool = False,
    ):
        super().__init__()

        # 0) Raw 64-dim → 16-dim patch projection
        self.input_proj = nn.Linear(input_dim, patch_length)

        # 1) LSTM backbone that expects `patch_length` features
        self.backbone = nn.LSTM(
            input_size     = patch_length,
            hidden_size    = hidden_size,
            num_layers     = num_layers,
            batch_first    = True,
            bidirectional  = bidirectional,
            dropout        = dropout if num_layers > 1 else 0.0,
        )
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        # 2) FC head
        lstm_out_dim = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.Linear(lstm_out_dim, out_dim)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (batch, seq_len, input_dim=64)
        returns: (batch, out_dim)
        """
        # project raw features to patch_length
        x_proj = self.input_proj(x)             # (B, seq_len, 16)

        # sequence modeling with LSTM
        out, _ = self.backbone(x_proj)          # (B, seq_len, lstm_out_dim)

        # take the last time-step representation
        feat = out[:, -1, :]                    # (B, lstm_out_dim)

        # downstream head
        return self.head(feat)                  # (B, out_dim)


## fine-tuning

In [20]:
# ──────────────────────────
# Shared hyper-parameters
# ──────────────────────────
INPUT_DIM     = 64     # raw feature dimension
PATCH_LENGTH  = 16     # dimension fed to every backbone
D_MODEL       = 64     # internal hidden size (GRU/LSTM/Transformer)
N_LAYERS      = 12     # stacked layers
HIDDEN_DIM    = 256    # head hidden dimension
OUT_DIM       = 64     # head output dimension
DROPOUT       = 0.1    # dropout for recurrent / transformer blocks
BIDIRECTIONAL = True   # use bidirectional RNNs
DEVICE        = "cuda"

# ──────────────────────────
# Model class catalog
# ──────────────────────────
MODEL_CATALOG = {
    "LWM_freeze_backbone"     : LWMWithHead,
    "LWM_pretrained_Fine_tune": LWMWithHead,
    "LWM_Fine_tune"           : LWMWithHead,
    "gru"                     : GRUWithHead,
    "RNN"                     : RNNWithHead,
    "LSTM"                    : LSTMWithHead,
    "Transformer"             : TransformerWithHead,
}

# ──────────────────────────
# Per-model constructor kwargs
# ──────────────────────────
MODEL_PARAMS = {
    # ── LWM variants ─────────────────────────────
    "LWM_freeze_backbone": {
        "input_dim"       : INPUT_DIM,
        "patch_length"    : PATCH_LENGTH,
        "d_model"         : D_MODEL,
        "max_len"         : PATCH_LENGTH + 1,
        "n_layers"        : N_LAYERS,
        "hidden_dim"      : HIDDEN_DIM,
        "out_dim"         : OUT_DIM,
        "freeze_backbone" : True,
        "checkpoint_path" : "./model_weights.pth",
        "device"          : DEVICE,
    },
    "LWM_pretrained_Fine_tune": {
        "input_dim"       : INPUT_DIM,
        "patch_length"    : PATCH_LENGTH,
        "d_model"         : D_MODEL,
        "max_len"         : PATCH_LENGTH + 1,
        "n_layers"        : N_LAYERS,
        "hidden_dim"      : HIDDEN_DIM,
        "out_dim"         : OUT_DIM,
        "freeze_backbone" : False,
        "checkpoint_path" : "./model_weights.pth",
        "device"          : DEVICE,
    },
    "LWM_Fine_tune": {
        "input_dim"       : INPUT_DIM,
        "patch_length"    : PATCH_LENGTH,
        "d_model"         : D_MODEL,
        "max_len"         : PATCH_LENGTH + 1,
        "n_layers"        : N_LAYERS,
        "hidden_dim"      : HIDDEN_DIM,
        "out_dim"         : OUT_DIM,
        "freeze_backbone" : False,
        "checkpoint_path" : None,
        "device"          : DEVICE,
    },

    # ── GRU (projected) ──────────────────────────
    "gru": {
        "input_dim"       : INPUT_DIM,     # 64 → project → 16
        "patch_length"    : PATCH_LENGTH,
        "d_model"         : D_MODEL,
        "n_layers"        : N_LAYERS,
        "bidirectional"   : BIDIRECTIONAL,
        "dropout"         : DROPOUT,
        "hidden_dim"      : HIDDEN_DIM,
        "out_dim"         : OUT_DIM,
        "freeze_backbone" : False,
    },

    # ── Vanilla RNN (projected) ──────────────────
    "RNN": {
        "input_dim"       : INPUT_DIM,
        "patch_length"    : PATCH_LENGTH,
        "hidden_size"     : D_MODEL,
        "num_layers"      : N_LAYERS,
        "bidirectional"   : BIDIRECTIONAL,
        "dropout"         : 0.0,
        "hidden_dim"      : HIDDEN_DIM,
        "out_dim"         : OUT_DIM,
        "freeze_backbone" : False,
    },

    # ── LSTM (projected) ─────────────────────────
    "LSTM": {
        "input_dim"       : INPUT_DIM,
        "patch_length"    : PATCH_LENGTH,
        "hidden_size"     : D_MODEL,
        "num_layers"      : N_LAYERS,
        "bidirectional"   : BIDIRECTIONAL,
        "dropout"         : DROPOUT,
        "hidden_dim"      : HIDDEN_DIM,
        "out_dim"         : OUT_DIM,
        "freeze_backbone" : False,
    },

    # ── Transformer (projected) ──────────────────
    "Transformer": {
        "input_dim"       : INPUT_DIM,
        "patch_length"    : PATCH_LENGTH,
        "d_model"         : D_MODEL,
        "n_heads"         : 4,
        "dim_ff"          : 256,
        "n_layers"        : N_LAYERS,
        "dropout"         : DROPOUT,
        "hidden_dim"      : HIDDEN_DIM,
        "out_dim"         : OUT_DIM,
        "max_len"         : PATCH_LENGTH + 1,
        "freeze_backbone" : False,
    },
}


## model evaluate

In [21]:
import torch
import torch.nn.functional as F

def rmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Root-Mean-Squared Error
    """
    return torch.sqrt(F.mse_loss(pred, target, reduction="mean"))   # √MSE

def nmse(pred: torch.Tensor, target: torch.Tensor, eps : float = 1e-12) -> torch.Tensor:
    """
    Normalized MSE  =  E[‖ŷ − y‖²] / E[‖y‖²]
    """
    # (B, …) → (B,)  
    mse_per_sample   = ((pred - target)**2).view(pred.size(0), -1).sum(dim=1)
    power_per_sample = (target**2).view(target.size(0), -1).sum(dim=1) + eps
    return (mse_per_sample / power_per_sample).mean()



In [22]:
def masked_evaluate(model, loader, device="cuda"):
    """
    Validation loop for IterableDataset.
    Returns average RMSE and NMSE over all samples.
    """
    model.eval()
    total_rmse, total_nmse, total_samples = 0.0, 0.0, 0

    with torch.no_grad():
        for input_ids, masked_pos, target in loader:
            # Move to device
            input_ids, masked_pos, target = (
                input_ids.to(device),
                masked_pos.to(device),
                target.to(device),
            )
            # Batch size
            bs = input_ids.size(0)

            # Forward
            pred = model(input_ids, masked_pos)

            # Accumulate batch metrics
            total_rmse    += rmse(pred, target).item() * bs
            total_nmse    += nmse(pred, target).item() * bs
            total_samples += bs

    # Compute averages
    return {
        "RMSE": total_rmse / total_samples,
        "NMSE": total_nmse / total_samples
    }

In [23]:
import inspect

def unmasked_evaluate(model, loader, device, patch_length=4):
    """
    Validation loop for IterableDataset.
    Computes and returns the average RMSE and NMSE over the dataset.
    """
    model.eval()
    total_rmse, total_nmse, total_samples = 0.0, 0.0, 0

    # Inspect the model's forward signature to determine if it requires a decoder input
    sig = inspect.signature(model.forward)
    needs_tgt = len(sig.parameters) >= 3  # True if forward(self, src, tgt, ...) exists

    with torch.no_grad():
        for input_ids, target in loader:
            # Move input and target tensors to the specified device
            input_ids = input_ids.to(device)
            target = target.to(device)

            if needs_tgt:
                # Transformer models: use the last `patch_length` time steps as decoder input
                tgt = input_ids[:, -patch_length:, :]
                pred = model(input_ids, tgt)
            else:
                # Single-input models (e.g., GRU, LSTM): only the source sequence is needed
                pred = model(input_ids)

            # Accumulate weighted metrics
            batch_size = input_ids.size(0)
            total_rmse += rmse(pred, target).item() * batch_size
            total_nmse += nmse(pred, target).item() * batch_size
            total_samples += batch_size

    # Calculate average RMSE and NMSE over all samples
    avg_rmse = total_rmse / total_samples
    avg_nmse = total_nmse / total_samples

    return {
        "RMSE": avg_rmse,
        "NMSE": avg_nmse
    }


# Model Training

In [24]:
"""
Unified training / validation script
------------------------------------
* Trains every architecture listed in MODEL_CATALOG
* Chooses masked / un-masked DataLoader automatically
* Reports per-epoch speed & validation scores
* Saves **best** and **last** checkpoints under ./checkpoints/
"""

# ─────────────────────────────────────────────
# 0) Globals and hyper-parameters
# ─────────────────────────────────────────────
device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion   = nn.MSELoss().to(device)

NUM_EPOCHS  = 30
LR          = 1e-4                         # learning-rate
CKPT_DIR    = Path("checkpoints")          # where *.pth files will be stored
CKPT_DIR.mkdir(exist_ok=True)

total_start = time.time()                  # wall-clock timer for *all* models
results     = {}                           # best-epoch NMSE(dB) for every model

# ─────────────────────────────────────────────
# 1) Train / validate each model
# ─────────────────────────────────────────────
for model_name, ModelCls in MODEL_CATALOG.items():

    print(f"\n=== Training {model_name} ===")
    model_args = MODEL_PARAMS[model_name]
    model      = ModelCls(**model_args).to(device)

    # collect only trainable parameters
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    if len(trainable_params) == 0:
        print(f"⚠️  '{model_name}' has no trainable parameters — skipping.")
        results[model_name] = float("nan")
        continue

    optimizer   = torch.optim.Adam(trainable_params, lr=LR)
    epoch_times = []                       # per-epoch training duration
    best_nmse   = float("inf")             # track the best val-NMSE

    # pick loaders / evaluation fn based on model family
    uses_mask  = model_name.startswith("LWM_") 
    tr_loader  = masked_train_loader if uses_mask else unmasked_train_loader
    val_loader = masked_val_loader  if uses_mask else unmasked_val_loader
    eval_fn    = masked_evaluate    if uses_mask else unmasked_evaluate

    # ── EPOCH LOOP ──────────────────────────
    for epoch in range(1, NUM_EPOCHS + 1):

        # ---------- TRAIN ----------
        t0 = time.time()
        model.train()
        run_loss = 0.0

        pbar = tqdm(tr_loader,
                    desc=f"[{model_name} {epoch:02d}/{NUM_EPOCHS}] train",
                    leave=False)

        for b, batch in enumerate(pbar, 1):
            if uses_mask:
                xb, mpos, yb = [x.to(device) for x in batch]
                pred = model(xb, mpos).squeeze(-1)
                
                
            else:
                xb, yb = [x.to(device) for x in batch]
                if model_name == "Transformer":
                    tgt = xb[:,4:,:]
                    pred = model(xb, tgt)
                else:
                    pred   = model(xb)
                    
                    

            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            run_loss += loss.item()
            if b % 100 == 0:
                pbar.set_postfix(train_loss=run_loss / b)

        epoch_times.append(time.time() - t0)
        avg_train_loss = run_loss / b

        # ---------- VALID ----------
        metrics   = eval_fn(model, val_loader, device)
        val_rmse  = metrics["RMSE"]
        val_nmse  = metrics["NMSE"]
        val_nmse_db = 10 * torch.log10(torch.tensor(val_nmse)).item()

        # save best checkpoint
        if val_nmse < best_nmse:
            best_nmse = val_nmse
            torch.save(
                model.state_dict(),
                CKPT_DIR / f"{model_name}_best.pth"
            )

        print(
            f"[{epoch:02d}/{NUM_EPOCHS}] "
            f"TrainLoss: {avg_train_loss:.4f}  "
            f"Val RMSE: {val_rmse:.4f}  "
            f"Val NMSE: {val_nmse:.4e}  "
            f"Val NMSE_dB: {val_nmse_db:.1f} dB  "
            f"TrainTime: {epoch_times[-1]:.2f}s"
        )

    # after all epochs – save *last* weights
    torch.save(
        model.state_dict(),
        CKPT_DIR / f"{model_name}_last.pth"
    )

    avg_ep_time = sum(epoch_times) / len(epoch_times)
    print(f"🕒 {model_name} – avg train time / epoch: {avg_ep_time:.2f}s")

    # store best NMSE_dB for the summary
    results[model_name] = 10 * math.log10(best_nmse)

# ─────────────────────────────────────────────
# 2) Summary
# ─────────────────────────────────────────────
print("\n=== Summary of best NMSE(dB) by model ===")
for name, nmse_db in results.items():
    print(f"{name:25s}: {nmse_db if not math.isnan(nmse_db) else 'skipped':>6}")

print(f"\nTotal training time for all models: {time.time() - total_start:.2f}s")



=== Training LWM_freeze_backbone ===
Model loaded successfully from ./model_weights.pth to cuda


[01/30] TrainLoss: 0.0223  Val RMSE: 0.0871  Val NMSE: 3.8447e-02  Val NMSE_dB: -14.2 dB  TrainTime: 373.19s


[02/30] TrainLoss: 0.0080  Val RMSE: 0.0840  Val NMSE: 3.4614e-02  Val NMSE_dB: -14.6 dB  TrainTime: 389.18s


[03/30] TrainLoss: 0.0073  Val RMSE: 0.0828  Val NMSE: 3.3010e-02  Val NMSE_dB: -14.8 dB  TrainTime: 378.48s


[04/30] TrainLoss: 0.0069  Val RMSE: 0.0813  Val NMSE: 3.1183e-02  Val NMSE_dB: -15.1 dB  TrainTime: 374.53s


...

[26/30] TrainLoss: 0.0014  Val RMSE: 0.0376  Val NMSE: 7.4742e-03  Val NMSE_dB: -21.3 dB  TrainTime: 717.96s


[27/30] TrainLoss: 0.0014  Val RMSE: 0.0382  Val NMSE: 7.5767e-03  Val NMSE_dB: -21.2 dB  TrainTime: 753.10s


[28/30] TrainLoss: 0.0013  Val RMSE: 0.0377  Val NMSE: 7.5185e-03  Val NMSE_dB: -21.2 dB  TrainTime: 740.63s


[29/30] TrainLoss: 0.0013  Val RMSE: 0.0375  Val NMSE: 7.4003e-03  Val NMSE_dB: -21.3 dB  TrainTime: 701.09s


[30/30] TrainLoss: 0.0013  Val RMSE: 0.0372  Val NMSE: 7.4422e-03  Val NMSE_dB: -21.3 dB  TrainTime: 786.03s
🕒 Transformer – avg train time / epoch: 722.31s

=== Summary of best NMSE(dB) by model ===
LWM_freeze_backbone      : -16.423087250923174
LWM_pretrained_Fine_tune : -18.64114847467836
LWM_Fine_tune            : -19.503833776827854
gru                      : -21.27034990986494
RNN                      : -21.44213286142136
LSTM                     : -20.797745537656915
Transformer              : -21.43605817300161

Total training time for all models: 96339.43s


## inference

In [26]:
# ─────────────────────────────────────────────
# 0)  Load the *best* checkpoints into `trained_models`
# ─────────────────────────────────────────────
CKPT_DIR = Path("checkpoints")                 # folder with *.pth files
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trained_models = {}
for name, ModelCls in MODEL_CATALOG.items():
    ckpt_path = CKPT_DIR / f"{name}_best.pth"
    if ckpt_path.exists():
        model = ModelCls(**MODEL_PARAMS[name])     # init on CPU
        model.load_state_dict(torch.load(ckpt_path, map_location="cpu"))
        trained_models[name] = model               # keep on CPU for now
    else:
        print(f"⚠️  {ckpt_path} not found — skipping this model.")

# ─────────────────────────────────────────────
# 1)  Pure-inference timing loop (no loss / labels)
# ─────────────────────────────────────────────
torch.backends.cudnn.benchmark = True           # let cuDNN pick fastest kernels
INFER_TIME = {}                                 # {model: (total, per_batch, per_sample)}

for name, model in trained_models.items():
    uses_mask      = name.startswith("LWM_")
    is_transformer = name.startswith("Transformer")   # covers Transformer & TransformerWithHead
    v_loader       = masked_val_loader if uses_mask else unmasked_val_loader

    model = model.to(device).eval()

    # ― Warm-up (one batch) to ramp GPU clocks and cache kernels
    with torch.no_grad():
        batch = next(iter(v_loader))
        if uses_mask:
            seq, mpos, _ = [x.to(device) for x in batch]
            _ = model(seq, mpos)
        elif is_transformer:
            seq, _ = [x.to(device) for x in batch]
            tgt    = seq[:, 4:, :]                 # same slice used during training
            _ = model(seq, tgt)
        else:
            seq, _ = [x.to(device) for x in batch]
            _ = model(seq)

    # ― Timed inference pass over the entire loader
    torch.cuda.synchronize()
    t0        = time.time()
    n_batches = 0
    n_samples = 0

    with torch.no_grad():
        for batch in v_loader:
            if uses_mask:
                seq, mpos, _ = [x.to(device) for x in batch]
                _  = model(seq, mpos)
                bs = seq.size(0)
            elif is_transformer:
                seq, _ = [x.to(device) for x in batch]
                tgt    = seq[:, 4:, :]
                _  = model(seq, tgt)
                bs = seq.size(0)
            else:
                seq, _ = [x.to(device) for x in batch]
                _  = model(seq)
                bs = seq.size(0)

            n_batches += 1
            n_samples += bs

    torch.cuda.synchronize()
    elapsed = time.time() - t0

    INFER_TIME[name] = (
        elapsed,                # total seconds
        elapsed / n_batches,    # seconds per batch
        elapsed / n_samples     # seconds per sample
    )

    print(f"⏱ {name:25s} | total {elapsed:6.2f}s  "
          f"| /batch {elapsed/n_batches*1e3:6.2f} ms  "
          f"| /sample {elapsed/n_samples*1e3:6.2f} ms")

# ─────────────────────────────────────────────
# 2)  Pretty summary table
# ─────────────────────────────────────────────
print("\n=== Inference-time summary ===")
header = f"{'model':25s} | {'total [s]':>9} | {'/batch [ms]':>12} | {'/sample [ms]':>13}"
print(header)
print("-" * len(header))
for n, (tot, pb, ps) in INFER_TIME.items():
    print(f"{n:25s} | {tot:9.2f} | {pb*1e3:12.2f} | {ps*1e3:13.2f}")


Model loaded successfully from ./model_weights.pth to cuda
Model loaded successfully from ./model_weights.pth to cuda
⏱ LWM_freeze_backbone       | total 105.33s  | /batch  26.58 ms  | /sample   0.83 ms
⏱ LWM_pretrained_Fine_tune  | total 100.56s  | /batch  25.38 ms  | /sample   0.79 ms
⏱ LWM_Fine_tune             | total  91.94s  | /batch  23.21 ms  | /sample   0.73 ms
⏱ gru                       | total  47.37s  | /batch  11.96 ms  | /sample   0.37 ms
⏱ RNN                       | total  45.68s  | /batch  11.53 ms  | /sample   0.36 ms
⏱ LSTM                      | total  51.65s  | /batch  13.04 ms  | /sample   0.41 ms
⏱ Transformer               | total 146.89s  | /batch  37.08 ms  | /sample   1.16 ms

=== Inference-time summary ===
model                     | total [s] |  /batch [ms] |  /sample [ms]
--------------------------------------------------------------------
LWM_freeze_backbone       |    105.33 |        26.58 |          0.83
LWM_pretrained_Fine_tune  |    100.56 |        2

## Compare trainable parameters

In [27]:
# ─────────────────────────────────────────────
# Report trainable parameters for every model
# ─────────────────────────────────────────────
print("\n=== Trainable parameters per model ===")
for name, ModelCls in MODEL_CATALOG.items():
    # instantiate model with its params (on CPU is fine for counting)
    model = ModelCls(**MODEL_PARAMS[name])
    count = count_trainable_params(model)
    print(f"{name:25s}: {count:,}")



=== Trainable parameters per model ===
Model loaded successfully from ./model_weights.pth to cuda
LWM_freeze_backbone      : 5,200
Model loaded successfully from ./model_weights.pth to cuda
LWM_pretrained_Fine_tune : 608,912
LWM_Fine_tune            : 601,744
gru                      : 860,240
RNN                      : 292,944
LSTM                     : 1,143,888
Transformer              : 1,408,208
